In [4]:
import pandas as pd

In [5]:
# 곡 정보 파일 불러오기
df = pd.read_csv('../SongTidy/FinalTidy/tidydata/song_tidy03.csv')

In [6]:
# 가수 정보 파일 불러오기
df2 = pd.read_csv('../Data/Data20180921/artist_info_combined_ver04.csv', encoding='euc-kr')

In [7]:
# 곡 정보와 가수 정보를 머지해준다.
song = pd.merge(df, df2, on='artist')

In [8]:
# 가사 분석에 용이하도록 <br>을 공백으로 바꿔준다.
song['lyrics'] = song['lyrics'].str.replace(r'<br>', ' ')

In [22]:
female = song[song['gender'] == '여']

In [23]:
male = song[song['gender'] == '남']

In [16]:
song_lyrics = song['lyrics'].tolist()

In [24]:
female_lyrics = female['lyrics'].tolist()
male_lyrics = male['lyrics'].tolist()

In [9]:
def preprocessing(text):
    # 개행문자 제거
    text =  text.strip('\t\n\r')
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [1]:
import sys
sys.path.append('../')

import soynlp
print(soynlp.__version__)

0.0.46


In [13]:
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2

corpus_path = song['lyrics']
sents = DoublespaceLineCorpus(corpus_path, iter_sent=True)

In [14]:
%%time
noun_extractor = LRNounExtractor_v2(verbose=True, extract_compound=True)
noun_extractor.train(sents)
nouns = noun_extractor.extract()

TypeError: __init__() got an unexpected keyword argument 'extract_compound'

In [19]:
from soynlp.noun import LRNounExtractor_v2
noun_extractor = LRNounExtractor_v2()
nouns = noun_extractor.train_extract(song_lyrics)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=1260, neg=1185, common=12
[Noun Extractor] counting eojeols
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained.
[Noun Extractor] batch prediction was completed for 34300 words
[Noun Extractor] checked compounds. discovered 3222 compounds
[Noun Extractor] 13303 nouns (3222 compounds) with min count=1
[Noun Extractor] 44.87 % eojeols are covered
[Noun Extractor] flushing ... done


In [25]:
nouns_f = noun_extractor.train_extract(female_lyrics)

[Noun Extractor] counting eojeols
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained.
[Noun Extractor] batch prediction was completed for 15076 words
[Noun Extractor] checked compounds. discovered 1080 compounds
[Noun Extractor] 5480 nouns (1080 compounds) with min count=1
[Noun Extractor] 39.82 % eojeols are covered
[Noun Extractor] flushing ... done


In [26]:
nouns_m = noun_extractor.train_extract(male_lyrics)

[Noun Extractor] counting eojeols
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained.
[Noun Extractor] batch prediction was completed for 26898 words
[Noun Extractor] checked compounds. discovered 1961 compounds
[Noun Extractor] 9824 nouns (1961 compounds) with min count=1
[Noun Extractor] 42.75 % eojeols are covered
[Noun Extractor] flushing ... done


In [158]:
# print(*nouns_f, sep='\n')

In [159]:
# print(*nouns_m, sep='\n')

In [170]:
dict(list(nouns.items())[0:10])

{'Mysterious': NounScore(frequency=25, score=1.0),
 'Strawberry': NounScore(frequency=9, score=1.0),
 'impossible': NounScore(frequency=8, score=1.0),
 'microphone': NounScore(frequency=13, score=1.0),
 'Fantastic': NounScore(frequency=25, score=1.0),
 'butterfly': NounScore(frequency=61, score=1.0),
 'Gentleman': NounScore(frequency=12, score=0.9230769230769231),
 'Christmas': NounScore(frequency=226, score=1.0),
 'fantastic': NounScore(frequency=41, score=1.0),
 'Beautiful': NounScore(frequency=420, score=1.0)}

In [105]:
pd.DataFrame(nouns_f, columns=['name', 'city'])

,name,city


In [171]:
dict(list(nouns_f.items())[0:10])

{'fantastic': NounScore(frequency=25, score=1.0),
 'Beautiful': NounScore(frequency=138, score=1.0),
 'Chocolate': NounScore(frequency=26, score=1.0),
 'Christmas': NounScore(frequency=129, score=0.9923076923076923),
 'Romantic': NounScore(frequency=7, score=1.0),
 'Perfect': NounScore(frequency=17, score=1.0),
 'Mystery': NounScore(frequency=11, score=1.0),
 'Fantasy': NounScore(frequency=32, score=1.0),
 'Dollars': NounScore(frequency=6, score=1.0),
 'Diamond': NounScore(frequency=15, score=0.5555555555555556)}

In [118]:
nouns_f_df = pd.DataFrame.from_dict(nouns_f, orient='index',
                       columns=['frequency', 'score'])

In [119]:
nouns_f_df.reset_index(level=0, inplace=True)

In [123]:
nouns_f_df = nouns_f_df.rename(index=str, columns={"index": "noun"})

In [124]:
nouns_f_df.head()

,noun,frequency,score
0,fantastic,25,1.000000
1,Beautiful,138,1.000000
2,Chocolate,26,1.000000
3,Christmas,129,0.992308
4,Romantic,7,1.000000


In [145]:
import re

In [153]:
nouns_f_df[nouns_f_df['noun'].str.contains(r'[이예][쁘쁜쁠뻐뻤]')].sort_values(by=['frequency'], axis=0, ascending=False)

,noun,frequency,score
2134,예뻐,188,0.622222
2261,이뻐,35,1.000000
2776,이쁜,19,0.791667
1821,예쁘다,15,0.454545
1350,예쁜걸,4,1.000000
989,예뻐보,3,0.500000
1047,예쁘장,3,0.500000
4434,예쁘장하기만,3,1.000000
4646,예뻐진다,3,0.622222
4871,너무예뻐,3,1.000000


In [152]:
nouns_m_df[nouns_m_df['noun'].str.contains(r'[이예][쁘쁜쁠뻐뻤]')].sort_values(by=['frequency'], axis=0, ascending=False)

,noun,frequency,score
4184,예뻐,483,0.890110
4700,이쁜,53,0.600000
6500,이뻐,21,0.333333
2695,예쁜데,19,1.000000
9187,이쁜이들,18,1.000000
8309,이쁜아가씨,2,1.000000
9509,이뻐지네,2,0.600000
9120,예뻐지고,1,0.923077
9132,예뻐너무,1,1.000000
9519,이쁜여자,1,0.600000


In [155]:
nouns_f_df[nouns_f_df['noun'].str.contains('여자')].sort_values(by=['frequency'], axis=0, ascending=False)

,noun,frequency,score
2384,여자,542,0.723757
708,여자들,103,1.000000
210,여자친구,20,1.000000
1380,여자라,14,1.000000
5205,여자인데,8,0.818182
1243,여자애,5,1.000000
5092,여자인지,4,1.000000
842,여자아,3,1.000000
316,여자애들,2,1.000000
1343,여자에,2,0.666667


In [156]:
nouns_f_df[nouns_f_df['noun'].str.contains('남자')].sort_values(by=['frequency'], axis=0, ascending=False)

,noun,frequency,score
1899,남자,539,0.486880
689,남자들,132,1.000000
223,남자친구,26,1.000000
4935,남자라면,10,1.000000
1300,남자에,6,0.857143
4884,남자이길,3,1.000000
5203,남자인지,2,1.000000
1686,남자애,1,0.333333
4446,남자없을때,1,1.000000
4470,남자친구문제,1,1.000000


In [154]:
nouns_m_df[nouns_m_df['noun'].str.contains('여자')].sort_values(by=['frequency'], axis=0, ascending=False)

,noun,frequency,score
3850,여자,728,0.540441
1199,여자들,144,1.000000
492,여자친구,69,0.909091
3297,내여자,8,1.000000
2711,여자에,5,0.416667
1924,한여자,4,0.333333
9231,여자일까,3,1.000000
9033,여자친군,3,0.800000
8831,여자이길,3,0.571429
663,여자애들,2,1.000000


In [157]:
nouns_m_df[nouns_m_df['noun'].str.contains('남자')].sort_values(by=['frequency'], axis=0, ascending=False)

,noun,frequency,score
3815,남자,863,0.641320
1200,남자들,162,1.000000
495,남자친구,28,1.000000
2389,남자라,24,0.312500
3365,남자인,18,0.333333
3352,그남자,5,1.000000
738,남자인걸,5,1.000000
1966,남자만,4,1.000000
2542,상남자,4,1.000000
9687,남자이길,4,0.641320


In [ ]:
nouns_m_df[nouns_m_df['noun'].str.contains(r'[이예][쁘쁜쁠뻐뻤]')]

In [125]:
nouns_f_df.sort_values(by=['frequency'], axis=0, ascending=False).head(10)

,noun,frequency,score
4117,너,8888,0.985256
945,you,7219,0.796799
4109,나,6190,0.876805
1885,사랑,5339,0.998619
213,love,3766,1.000000
3537,나를,3652,0.333333
2759,내가,3375,1.000000
4124,말,2856,0.737593
3447,너의,2784,0.993931
1994,그대,2681,0.683507


In [92]:
nouns_f_df.shape

(5480, 2)

In [88]:
nouns_m_df = pd.DataFrame.from_dict(nouns_m, orient='index',
                        columns=['frequency', 'score'])

In [132]:
nouns_m_df.reset_index(level=0, inplace=True)
nouns_m_df = nouns_m_df.rename(index=str, columns={"index": "noun"})

In [133]:
nouns_m_df.sort_values(by=['frequency'], axis=0, ascending=False).head(10)

,noun,frequency,score
7527,너,15009,0.986494
7522,나,13531,0.350442
2436,you,12505,0.778110
3766,사랑,7601,0.997932
5263,내가,7104,0.666667
5223,우리,4677,0.968912
4089,그대,4650,0.541591
5282,이제,4624,0.979922
6002,내게,4527,0.890244
3912,너무,3668,1.000000


In [93]:
nouns_m_df.shape

(9824, 2)

In [134]:
nouns_fm = pd.merge(nouns_f_df, nouns_m_df, on=['noun'], how='outer')

In [136]:
nouns_fm.head()

,noun,frequency_x,score_x,frequency_y,score_y
0,fantastic,25.0,1.000000,NaN,NaN
1,Beautiful,138.0,1.000000,NaN,NaN
2,Chocolate,26.0,1.000000,NaN,NaN
3,Christmas,129.0,0.992308,96.0,1.0
4,Romantic,7.0,1.000000,2.0,1.0


In [137]:
nouns_fm = nouns_fm.rename(index=str, columns={"frequency_x": "frequency_f", "frequency_y": "frequency_m", 
                                              "score_x": "score_f", "score_y": "score_m"})

In [140]:
nouns_fm.sort_values(by=['frequency_f'], ascending = False).head(10)

,noun,frequency_f,score_f,frequency_m,score_m
4117,너,8888.0,0.985256,15009.0,0.986494
945,you,7219.0,0.796799,12505.0,0.778110
4109,나,6190.0,0.876805,13531.0,0.350442
1885,사랑,5339.0,0.998619,7601.0,0.997932
213,love,3766.0,1.000000,NaN,NaN
3537,나를,3652.0,0.333333,NaN,NaN
2759,내가,3375.0,1.000000,7104.0,0.666667
4124,말,2856.0,0.737593,3228.0,0.564236
3447,너의,2784.0,0.993931,NaN,NaN
1994,그대,2681.0,0.683507,4650.0,0.541591


In [138]:
nouns_fm[nouns_fm['frequency_m'].isnull()].sort_values(by=['frequency_f'], ascending = False).head(10)

,noun,frequency_f,score_f,frequency_m,score_m
213,love,3766.0,1.000000,NaN,NaN
3537,나를,3652.0,0.333333,NaN,NaN
3447,너의,2784.0,0.993931,NaN,NaN
3075,자꾸,1762.0,1.000000,NaN,NaN
3230,up,1685.0,1.000000,NaN,NaN
1848,boy,1507.0,0.918902,NaN,NaN
2047,in,1149.0,0.904013,NaN,NaN
3268,말고,677.0,0.998525,NaN,NaN
3138,우린,615.0,0.400000,NaN,NaN
2434,나만,584.0,0.333333,NaN,NaN


In [139]:
nouns_fm[nouns_fm['frequency_f'].isnull()].sort_values(by=['frequency_m'], ascending = False).head(10)

,noun,frequency_f,score_f,frequency_m,score_m
9217,내게,NaN,NaN,4527.0,0.890244
8423,니가,NaN,NaN,3497.0,0.666667
5869,Baby,NaN,NaN,2857.0,0.990638
8965,너와,NaN,NaN,2728.0,0.982709
8357,go,NaN,NaN,2596.0,0.375253
9908,너는,NaN,NaN,1896.0,0.997895
6765,너에게,NaN,NaN,1629.0,0.992083
8261,너만,NaN,NaN,1465.0,0.473684
6400,사랑을,NaN,NaN,1401.0,0.600000
7661,get,NaN,NaN,1395.0,0.931864


In [172]:
# print(*nouns, sep='\n')

In [29]:
noun_extractor.lrgraph.get_l('었다고')

[('잊', 18),
 ('늦', 10),
 ('사랑했', 8),
 ('있', 6),
 ('아니', 6),
 ('싶', 5),
 ('없', 4),
 ('행복했', 4),
 ('멀', 3),
 ('죽', 3)]

In [31]:
noun_extractor.decompose_compound('찰리채플린')

('찰리채플린',)

In [52]:
from soynlp.utils import LRGraph
from soynlp.utils import EojeolCounter

eojeol_counter_f = EojeolCounter(female_lyrics, min_count=1, max_length=15)
lrgraph_f = eojeol_counter_f.to_lrgraph(l_max_length=10, r_max_length=9)

In [54]:
eojeol_counter_m = EojeolCounter(male_lyrics, min_count=1, max_length=15)
lrgraph_m = eojeol_counter_m.to_lrgraph(l_max_length=10, r_max_length=9)

In [76]:
lrgraph_f.get_l('었다고')

[('잊', 13),
 ('있', 13),
 ('없', 5),
 ('늦', 5),
 ('장난이', 3),
 ('재미있', 3),
 ('들', 2),
 ('붙', 2),
 ('싶', 2),
 ('시간이', 2)]

In [75]:
lrgraph_m.get_l('었다고')

[('잊', 18),
 ('늦', 10),
 ('사랑했', 8),
 ('있', 6),
 ('아니', 6),
 ('싶', 5),
 ('없', 4),
 ('행복했', 4),
 ('멀', 3),
 ('죽', 3)]